# ASL Recognition with Transformer Architecture - Ultra-Optimized Pipeline

## 🚀 Enhanced Transformer with Advanced Features

This notebook implements a state-of-the-art ASL recognition system using:

- **🎯 Transformer Architecture**: Multi-head attention for sequence modeling
- **🔧 Advanced GPU Optimizations**: Mixed precision, model compilation, speed tracking
- **📊 Comprehensive Monitoring**: tqdm progress bars, real-time metrics
- **💾 Smart Checkpointing**: Auto-save best models with full state
- **🔄 Data Augmentation**: Spatial and temporal transformations
- **📈 Rich Visualizations**: Training curves, confusion matrices, per-class analysis
- **⚡ Ultra-Fast Training**: All optimizations from TGCN pipeline + Transformer power

### Architecture Overview

- **Input**: MediaPipe keypoint sequences (seq_len, 553, 3) - FIXED 'nodes' key
- **Normalization**: Advanced SimpleNormalizer with proper handling
- **Model**: Transformer encoder with positional encoding
- **Features**: All advanced training features + GPU acceleration
- **Target**: Match/exceed TGCN performance with Transformer efficiency


## 📚 Import All Required Libraries + GPU Optimizations


In [28]:
# Core libraries
# Fix the imports section - add missing autocast import
# Core libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast  # Added autocast import

# Progress bars and timing
from tqdm.auto import tqdm, trange
import time

# Data handling and visualization
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎯 Using device: {device}")
print(f"🚀 PyTorch version: {torch.__version__}")
print(f"⚡ CUDA available: {torch.cuda.is_available()}")

# 🚀 Enable Windows-compatible GPU optimizations
if torch.cuda.is_available():
    print("\n🚀 Enabling Windows-Compatible GPU Optimizations:")
    torch.backends.cudnn.benchmark = True  # Optimize cuDNN
    torch.backends.cuda.matmul.allow_tf32 = True  # Enable TF32
    torch.backends.cudnn.allow_tf32 = True
    print("  ✅ cuDNN benchmark enabled")
    print("  ✅ TF32 enabled for faster matmul")
    
    # Check GPU memory
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"  📊 GPU Memory: {gpu_memory:.1f} GB")
else:
    print("❌ No GPU detected - training will be slower on CPU")

🎯 Using device: cuda
🚀 PyTorch version: 2.7.0+cu118
⚡ CUDA available: True

🚀 Enabling Windows-Compatible GPU Optimizations:
  ✅ cuDNN benchmark enabled
  ✅ TF32 enabled for faster matmul
  📊 GPU Memory: 4.0 GB


## 📊 Configuration and Paths


In [38]:
# Data paths
DATA_DIR = r'f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints'
CHECKPOINT_DIR = r'f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\src\checkpoints'
MODEL_SAVE_PATH = os.path.join(CHECKPOINT_DIR, 'best_transformer_model.pth')

# Create directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# 🚀 WINDOWS-COMPATIBLE GPU CONFIGURATION FOR SPEED 🚀
CONFIG = {
    # Data parameters - OPTIMIZED FOR SPEED
    'max_seq_len': 50,          # Sequence length for padding/truncation
    'num_nodes': 553,           # 33 pose + 42 hands + 478 face landmarks
    'num_features': 3,          # x, y, z coordinates
    'max_classes': 50,          # 🚀 Limit classes for faster training
    'test_size': 0.2,           # Train/test split ratio
    'val_size': 0.15,           # Validation split ratio
    'batch_size': 32,           # 🚀 Optimized batch size for GPU efficiency
    
    # Model architecture - TRANSFORMER SPECIFIC
    'd_model': 128,             # Transformer model dimension
    'nhead': 8,                 # Number of attention heads
    'num_layers': 4,            # Number of transformer layers
    'dropout': 0.1,             # Dropout rate
    'dim_feedforward': 512,     # Feedforward dimension
    
    # Training parameters - OPTIMIZED FOR SPEED
    'num_epochs': 100,          # Maximum training epochs
    'learning_rate': 0.001,     # Initial learning rate
    'weight_decay': 1e-4,       # L2 regularization
    'patience': 15,             # Early stopping patience
    'min_lr': 1e-6,             # Minimum learning rate
    
    # 🚀 WINDOWS-COMPATIBLE GPU OPTIMIZATION SETTINGS 🚀
    'use_mixed_precision': True,  # 🚀 Enable AMP for 2x speed
    'pin_memory': True,           # 🚀 Faster CPU->GPU transfer
    'non_blocking': True,         # 🚀 Async GPU transfers
    'compile_model': False,       # 🚀 DISABLED - Triton not available on Windows
    'use_channels_last': False,   # 🚀 DISABLED - Can cause issues on some Windows setups
    'gradient_accumulation': 1,   # No gradient accumulation for speed
    
    # Data augmentation - SIMPLIFIED FOR SPEED
    'use_augmentation': True,     # Enable data augmentation
    'aug_probability': 0.2,       # 🚀 Probability of applying augmentation
    'spatial_aug_strength': 0.05, # Spatial augmentation strength
    'temporal_aug_strength': 0.1, # Temporal augmentation strength
    
    # DataLoader optimization 🚀
    'num_workers': 0,             # 🚀 Set to 0 for Windows/Jupyter compatibility
    'prefetch_factor': 2,         # 🚀 Prefetch batches
    'persistent_workers': False,  # 🚀 Don't keep workers alive (Windows compatibility)
    
    # 🚀 MISSING CONFIGURATION KEYS - ADDED 🚀
    'save_checkpoints': True,     # Enable model checkpointing
    'checkpoint_dir': CHECKPOINT_DIR,  # Checkpoint directory
    'scheduler_type': 'OneCycleLR',    # Learning rate scheduler type
    'optimizer_type': 'AdamW',         # Optimizer type
    'checkpoint_frequency': 10,        # Save checkpoint every N epochs
}

# Calculate input dimension
CONFIG['input_dim'] = CONFIG['num_nodes'] * CONFIG['num_features']

print("📋 Ultra-Optimized Transformer Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

print(f"\n📁 Data directory: {DATA_DIR}")
print(f"💾 Checkpoint directory: {CHECKPOINT_DIR}")

# 🚀 Adjust config based on GPU memory
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    if gpu_memory < 6:
        print("  ⚠️  Limited GPU memory detected - using conservative settings")
        CONFIG['batch_size'] = min(CONFIG['batch_size'], 24)
        CONFIG['d_model'] = min(CONFIG['d_model'], 96)
else:
    CONFIG['batch_size'] = 16
    CONFIG['use_mixed_precision'] = False

📋 Ultra-Optimized Transformer Configuration:
  max_seq_len: 50
  num_nodes: 553
  num_features: 3
  max_classes: 50
  test_size: 0.2
  val_size: 0.15
  batch_size: 32
  d_model: 128
  nhead: 8
  num_layers: 4
  dropout: 0.1
  dim_feedforward: 512
  num_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  patience: 15
  min_lr: 1e-06
  use_mixed_precision: True
  pin_memory: True
  non_blocking: True
  compile_model: False
  use_channels_last: False
  gradient_accumulation: 1
  use_augmentation: True
  aug_probability: 0.2
  spatial_aug_strength: 0.05
  temporal_aug_strength: 0.1
  num_workers: 0
  prefetch_factor: 2
  persistent_workers: False
  save_checkpoints: True
  checkpoint_dir: f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\src\checkpoints
  scheduler_type: OneCycleLR
  optimizer_type: AdamW
  checkpoint_frequency: 10
  input_dim: 1659

📁 Data directory: f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints
💾 Checkpoint dir

## Advanced Data Normalization Class


## 🎯 Hyperparameter Tuning for 69% Target Accuracy

Based on your successful 69% Transformer implementation, let's implement targeted hyperparameter configurations.
We'll test multiple proven configurations for sequence classification tasks.


In [39]:
# 🎯 HYPERPARAMETER TUNING - TARGETING 69% ACCURACY 🎯
# Based on successful Transformer implementations for sequence classification

print("🎯 Implementing Hyperparameter Tuning for Target ~69% Accuracy")
print("=" * 70)

# Save original config
ORIGINAL_CONFIG = CONFIG.copy()

# 📊 OPTIMIZED CONFIGURATIONS - Multiple proven setups
OPTIMIZED_CONFIGS = {
    'config_1_powerful': {
        # More powerful model with proven architecture
        'd_model': 256,              # 🚀 Increased from 96 -> 256
        'nhead': 8,                  # Keep 8 heads (256/8 = 32 per head)
        'num_layers': 6,             # 🚀 Increased layers for better representation
        'dropout': 0.2,              # 🚀 Increased dropout for regularization  
        'dim_feedforward': 1024,     # 🚀 Larger feedforward (4x d_model)
        'learning_rate': 0.0005,     # 🚀 Lower LR for stability
        'batch_size': 16,            # 🚀 Smaller batch for larger model
        'weight_decay': 1e-3,        # 🚀 Stronger regularization
        'patience': 20,              # More patience for convergence
    },
    
    'config_2_balanced': {
        # Balanced configuration - sweet spot
        'd_model': 192,              # 🚀 Increased from 96 -> 192
        'nhead': 8,                  # 192/8 = 24 per head
        'num_layers': 4,             # Keep 4 layers
        'dropout': 0.15,             # 🚀 Slight increase in dropout
        'dim_feedforward': 768,      # 4x d_model
        'learning_rate': 0.0008,     # 🚀 Slightly lower LR
        'batch_size': 20,            # Medium batch size
        'weight_decay': 5e-4,        # Balanced regularization
        'patience': 18,
    },
    
    'config_3_efficient': {
        # Efficient but effective configuration
        'd_model': 128,              # Back to original 128
        'nhead': 8,                  # 128/8 = 16 per head
        'num_layers': 3,             # 🚀 Fewer layers, focus on quality
        'dropout': 0.3,              # 🚀 Higher dropout for generalization
        'dim_feedforward': 512,      # 4x d_model
        'learning_rate': 0.0012,     # 🚀 Slightly higher LR for faster convergence
        'batch_size': 28,            # Larger batch for efficiency
        'weight_decay': 2e-4,        # Light regularization
        'patience': 15,
    },
    
    'config_4_deep': {
        # Deeper model with aggressive regularization
        'd_model': 160,              # 🚀 Custom dimension
        'nhead': 10,                 # 🚀 More heads (160/10 = 16 per head)
        'num_layers': 8,             # 🚀 Deep model
        'dropout': 0.25,             # Strong regularization
        'dim_feedforward': 640,      # 4x d_model
        'learning_rate': 0.0003,     # 🚀 Low LR for deep model stability
        'batch_size': 12,            # Small batch for deep model
        'weight_decay': 1e-3,        # Strong weight decay
        'patience': 25,              # Extra patience for deep model
    }
}

# 🚀 ADVANCED TRAINING OPTIMIZATIONS
ADVANCED_OPTIMIZATIONS = {
    'scheduler_type': 'CosineAnnealingWarmRestarts',  # 🚀 Better than OneCycleLR for transformers
    'warmup_epochs': 5,                               # 🚀 Learning rate warmup
    'label_smoothing': 0.15,                         # 🚀 Better than 0.1 for transformers
    'gradient_clip_norm': 0.5,                       # 🚀 Tighter gradient clipping
    'use_layer_norm': True,                          # 🚀 Additional layer normalization
    'positional_encoding_type': 'learnable',         # 🚀 Learnable vs fixed positional encoding
}

print("📊 Available Optimized Configurations:")
for config_name, config in OPTIMIZED_CONFIGS.items():
    estimated_params = config['d_model'] * config['d_model'] * config['num_layers'] * 4  # Rough estimate
    print(f"\n  🔧 {config_name}:")
    print(f"    📐 d_model: {config['d_model']}, layers: {config['num_layers']}, heads: {config['nhead']}")
    print(f"    📚 LR: {config['learning_rate']}, batch_size: {config['batch_size']}")
    print(f"    💾 Est. params: ~{estimated_params/1000:.0f}K")

# 🎯 SELECT OPTIMAL CONFIGURATION
print("\n🎯 Selecting Optimal Configuration...")

# Check GPU memory and select best config
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"📊 Available GPU Memory: {gpu_memory:.1f} GB")
    
    if gpu_memory >= 8:
        selected_config = 'config_1_powerful'
        print("✅ Selected: POWERFUL configuration (8GB+ GPU)")
    elif gpu_memory >= 6:
        selected_config = 'config_2_balanced' 
        print("✅ Selected: BALANCED configuration (6GB+ GPU)")
    elif gpu_memory >= 4:
        selected_config = 'config_3_efficient'
        print("✅ Selected: EFFICIENT configuration (4GB+ GPU)")
    else:
        selected_config = 'config_4_deep'
        print("✅ Selected: DEEP configuration (optimized for limited memory)")
else:
    selected_config = 'config_3_efficient'
    print("✅ Selected: EFFICIENT configuration (CPU fallback)")

# 🚀 APPLY SELECTED CONFIGURATION
optimal_config = OPTIMIZED_CONFIGS[selected_config]
print(f"\n🔄 Applying {selected_config} configuration...")

# Update CONFIG with optimal settings
for key, value in optimal_config.items():
    CONFIG[key] = value
    print(f"  🔧 {key}: {CONFIG[key]}")

# Apply advanced optimizations
for key, value in ADVANCED_OPTIMIZATIONS.items():
    CONFIG[key] = value
    print(f"  ⚡ {key}: {CONFIG[key]}")

# 🎯 CALCULATE MODEL COMPLEXITY
input_dim = CONFIG['num_nodes'] * CONFIG['num_features']
estimated_total_params = (
    input_dim * CONFIG['d_model'] +  # Input projection
    CONFIG['d_model'] * CONFIG['d_model'] * CONFIG['nhead'] * CONFIG['num_layers'] * 3 +  # Attention
    CONFIG['d_model'] * CONFIG['dim_feedforward'] * CONFIG['num_layers'] * 2 +  # FFN
    CONFIG['d_model'] * CONFIG['max_classes']  # Classification head
)

print(f"\n📊 OPTIMIZED MODEL SPECIFICATIONS:")
print(f"  🧠 Estimated Parameters: ~{estimated_total_params/1000000:.2f}M")
print(f"  💾 Estimated Model Size: ~{estimated_total_params * 4 / 1024 / 1024:.1f}MB")
print(f"  🎯 Target Accuracy: ~69% (matching your successful implementation)")
print(f"  ⚡ Expected Training Time: ~{CONFIG['num_epochs'] * len(range(425)) // CONFIG['batch_size'] * 2:.0f}s per epoch")

print("\n🚀 Configuration optimization complete! Ready for high-performance training.")

🎯 Implementing Hyperparameter Tuning for Target ~69% Accuracy
📊 Available Optimized Configurations:

  🔧 config_1_powerful:
    📐 d_model: 256, layers: 6, heads: 8
    📚 LR: 0.0005, batch_size: 16
    💾 Est. params: ~1573K

  🔧 config_2_balanced:
    📐 d_model: 192, layers: 4, heads: 8
    📚 LR: 0.0008, batch_size: 20
    💾 Est. params: ~590K

  🔧 config_3_efficient:
    📐 d_model: 128, layers: 3, heads: 8
    📚 LR: 0.0012, batch_size: 28
    💾 Est. params: ~197K

  🔧 config_4_deep:
    📐 d_model: 160, layers: 8, heads: 10
    📚 LR: 0.0003, batch_size: 12
    💾 Est. params: ~819K

🎯 Selecting Optimal Configuration...
📊 Available GPU Memory: 4.0 GB
✅ Selected: DEEP configuration (optimized for limited memory)

🔄 Applying config_4_deep configuration...
  🔧 d_model: 160
  🔧 nhead: 10
  🔧 num_layers: 8
  🔧 dropout: 0.25
  🔧 dim_feedforward: 640
  🔧 learning_rate: 0.0003
  🔧 batch_size: 12
  🔧 weight_decay: 0.001
  🔧 patience: 25
  ⚡ scheduler_type: CosineAnnealingWarmRestarts
  ⚡ warmup_ep

In [40]:
class SimpleNormalizer:
    """Advanced normalizer with proper keypoint handling and validation"""
    
    def __init__(self):
        self.center_x = None
        self.center_y = None
        self.scale = None
        self.fitted = False
        # Add mean and std for compatibility with deployment code
        self.mean = None
        self.std = None
        
    def fit(self, sequences):
        """Fit normalizer on training data sequences"""
        print("🔧 Fitting normalizer on training data...")
        
        all_points = []
        valid_count = 0
        total_count = 0
        
        for seq in tqdm(sequences, desc="Processing sequences for normalization"):
            if seq.size > 0:
                # Flatten sequence to get all points
                flat_seq = seq.reshape(-1, seq.shape[-1])
                
                # Only use x, y coordinates for normalization (ignore z if present)
                if flat_seq.shape[1] >= 2:
                    xy_points = flat_seq[:, :2]
                    
                    # Filter out zero/invalid points
                    valid_mask = ~(np.isnan(xy_points).any(axis=1) | 
                                  np.isinf(xy_points).any(axis=1) |
                                  (np.abs(xy_points) < 1e-6).all(axis=1))
                    
                    valid_points = xy_points[valid_mask]
                    if len(valid_points) > 0:
                        all_points.append(valid_points)
                        valid_count += len(valid_points)
                    
                    total_count += len(xy_points)
        
        print(f"  📊 Valid keypoints: {valid_count:,} / {total_count:,} ({100*valid_count/max(total_count,1):.1f}%)")
        
        if all_points:
            all_points = np.vstack(all_points)
            
            # Use robust statistics for better normalization
            self.center_x = np.median(all_points[:, 0])
            self.center_y = np.median(all_points[:, 1])
            
            # Calculate scale using interquartile range for robustness
            distances = np.sqrt((all_points[:, 0] - self.center_x)**2 + 
                              (all_points[:, 1] - self.center_y)**2)
            self.scale = np.percentile(distances, 95)
            
            if self.scale == 0 or np.isnan(self.scale):
                self.scale = 1.0
                
            # Calculate mean and std for deployment compatibility
            self.mean = np.array([self.center_x, self.center_y, 0.0])  # x, y, z
            self.std = np.array([self.scale, self.scale, 1.0])  # x, y, z
                
            self.fitted = True
            
            print(f"  📈 Normalization parameters:")
            print(f"    Center: ({self.center_x:.3f}, {self.center_y:.3f})")
            print(f"    Scale: {self.scale:.3f}")
        else:
            print("  ⚠️  No valid points found, using default normalization")
            self.center_x, self.center_y, self.scale = 0.0, 0.0, 1.0
            self.mean = np.array([0.0, 0.0, 0.0])
            self.std = np.array([1.0, 1.0, 1.0])
            self.fitted = True
    
    def transform(self, sequence):
        """Transform a sequence using fitted parameters"""
        if not self.fitted:
            raise ValueError("Normalizer must be fitted before transform")
            
        if sequence.size == 0:
            return sequence
            
        normalized = sequence.copy().astype(np.float32)
        
        # Normalize x, y coordinates
        if normalized.shape[-1] >= 2:
            normalized[..., 0] = (normalized[..., 0] - self.center_x) / self.scale
            normalized[..., 1] = (normalized[..., 1] - self.center_y) / self.scale
        
        # Handle NaN and infinite values
        normalized = np.nan_to_num(normalized, nan=0.0, posinf=2.0, neginf=-2.0)
        
        # Clip to reasonable range
        normalized = np.clip(normalized, -3, 3)
        
        return normalized
    
    def fit_transform(self, sequences):
        """Fit and transform in one step"""
        self.fit(sequences)
        return [self.transform(seq) for seq in sequences]

# Data augmentation functions
def apply_spatial_augmentation(sequence, scale_range=0.1, translation_range=0.05, 
                             rotation_range=0.1, noise_std=0.01):
    """Apply spatial augmentations to a sequence"""
    aug_seq = sequence.copy()
    
    if aug_seq.size == 0:
        return aug_seq
    
    # Random scaling
    if scale_range > 0:
        scale_factor = 1 + np.random.uniform(-scale_range, scale_range)
        aug_seq[..., :2] *= scale_factor
    
    # Random translation
    if translation_range > 0:
        tx = np.random.uniform(-translation_range, translation_range)
        ty = np.random.uniform(-translation_range, translation_range)
        aug_seq[..., 0] += tx
        aug_seq[..., 1] += ty
    
    # Random rotation (simple 2D rotation)
    if rotation_range > 0:
        angle = np.random.uniform(-rotation_range, rotation_range)
        cos_a, sin_a = np.cos(angle), np.sin(angle)
        
        x_coords = aug_seq[..., 0].copy()
        y_coords = aug_seq[..., 1].copy()
        
        aug_seq[..., 0] = x_coords * cos_a - y_coords * sin_a
        aug_seq[..., 1] = x_coords * sin_a + y_coords * cos_a
    
    # Add noise
    if noise_std > 0:
        noise = np.random.normal(0, noise_std, aug_seq.shape)
        aug_seq += noise
    
    return aug_seq

def apply_temporal_augmentation(sequence, speed_range=0.2):
    """Apply temporal augmentations (speed changes)"""
    if sequence.size == 0 or len(sequence) < 2:
        return sequence
    
    # Random speed change
    speed_factor = 1 + np.random.uniform(-speed_range, speed_range)
    new_length = max(1, int(len(sequence) * speed_factor))
    
    # Interpolate to new length
    indices = np.linspace(0, len(sequence) - 1, new_length)
    
    aug_seq = np.zeros((new_length, sequence.shape[1], sequence.shape[2]))
    
    for i in range(sequence.shape[1]):  # For each keypoint
        for j in range(sequence.shape[2]):  # For each coordinate
            aug_seq[:, i, j] = np.interp(indices, np.arange(len(sequence)), sequence[:, i, j])
    
    return aug_seq

## Dataset Class


In [41]:
import os
import glob
from sklearn.model_selection import train_test_split

class ASLKeypointDataset(Dataset):
    """Enhanced ASL Dataset with proper 'nodes' key handling and validation"""
    
    def __init__(self, data_dir, max_seq_len=50, split='train', test_size=0.2, 
                 random_state=42, use_subset=False, max_classes=None, normalizer=None,
                 use_augmentation=False):
        self.data_dir = data_dir
        self.max_seq_len = max_seq_len
        self.split = split
        self.normalizer = normalizer
        self.use_augmentation = use_augmentation
        
        self.data = []
        self.labels = []
        self.word_to_idx = {}
        self.idx_to_word = {}
        self.class_names = []
        
        print(f"🔄 Loading {split} dataset from {data_dir}...")
        self._load_and_split_data(test_size, random_state, use_subset, max_classes)
        
    def _load_and_split_data(self, test_size, random_state, use_subset, max_classes):
        """Load data and create train/test splits"""
        
        # Find all word directories
        if not os.path.exists(self.data_dir):
            raise FileNotFoundError(f"Data directory not found: {self.data_dir}")
            
        word_dirs = [d for d in os.listdir(self.data_dir) 
                    if os.path.isdir(os.path.join(self.data_dir, d))]
        word_dirs = sorted(word_dirs)
        
        if use_subset and max_classes:
            word_dirs = word_dirs[:max_classes]
            print(f"  📊 Using subset: {len(word_dirs)} classes")
        
        # Collect all data first
        all_data = []
        all_labels = []
        current_idx = 0
        
        for word in tqdm(word_dirs, desc="Loading classes"):
            word_dir = os.path.join(self.data_dir, word)
            npz_files = glob.glob(os.path.join(word_dir, "*.npz"))
            
            if len(npz_files) == 0:
                continue
                
            # Add word to mapping
            self.word_to_idx[word] = current_idx
            self.idx_to_word[current_idx] = word
            self.class_names.append(word)
            
            valid_samples = 0
            for npz_file in npz_files:
                try:
                    data = np.load(npz_file)
                    
                    # Look for 'nodes' key (new format) or 'pose' key (old format)
                    if 'nodes' in data:
                        keypoints = data['nodes']
                    elif 'pose' in data:
                        keypoints = data['pose']
                        print(f"  ⚠️  Using 'pose' key for {npz_file} (should be 'nodes')")
                    else:
                        print(f"  ❌ No valid keypoint data in {npz_file}")
                        continue
                    
                    # Validate shape
                    if keypoints.ndim == 3 and keypoints.shape[0] > 0:
                        # Apply normalization if provided
                        if self.normalizer:
                            keypoints = self.normalizer.transform(keypoints)
                        
                        # Pad or truncate to target length
                        processed = self._pad_or_truncate(keypoints)
                        
                        all_data.append(processed)
                        all_labels.append(current_idx)
                        valid_samples += 1
                    else:
                        print(f"  ⚠️  Invalid keypoint shape {keypoints.shape} in {npz_file}")
                        
                except Exception as e:
                    print(f"  ❌ Error loading {npz_file}: {e}")
                    continue
            
            print(f"  📁 {word}: {valid_samples} valid samples")
            current_idx += 1
        
        # Convert to arrays
        all_data = np.array(all_data)
        all_labels = np.array(all_labels)
        
        print(f"  📊 Total samples: {len(all_data)}")
        print(f"  📊 Total classes: {len(self.class_names)}")
        
        # Create train/test split
        if len(all_data) > 0:
            train_data, test_data, train_labels, test_labels = train_test_split(
                all_data, all_labels, test_size=test_size, random_state=random_state,
                stratify=all_labels if len(np.unique(all_labels)) > 1 else None
            )
            
            if self.split == 'train':
                self.data = train_data
                self.labels = train_labels
                print(f"  ✅ Train split: {len(self.data)} samples")
            else:
                self.data = test_data  
                self.labels = test_labels
                print(f"  ✅ Test split: {len(self.data)} samples")
        else:
            raise ValueError("No valid samples found in dataset")
            
        self.num_classes = len(self.class_names)
        
    def _pad_or_truncate(self, sequence):
        """Pad or truncate sequence to target length"""
        if len(sequence) >= self.max_seq_len:
            # Truncate
            return sequence[:self.max_seq_len]
        else:
            # Pad with zeros
            padding = np.zeros((self.max_seq_len - len(sequence), 
                              sequence.shape[1], sequence.shape[2]))
            return np.vstack([sequence, padding])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sequence = self.data[idx].copy()
        label = self.labels[idx]
        
        # Apply data augmentation if enabled
        if self.use_augmentation and self.split == 'train':
            if np.random.random() < CONFIG['aug_probability']:
                # Apply spatial augmentation
                if np.random.random() < 0.5:
                    sequence = apply_spatial_augmentation(
                        sequence,
                        scale_range=CONFIG['spatial_aug_strength'],
                        translation_range=CONFIG['spatial_aug_strength']
                    )
                
                # Apply temporal augmentation
                if np.random.random() < 0.3:
                    aug_seq = apply_temporal_augmentation(
                        sequence, speed_range=CONFIG['temporal_aug_strength']
                    )
                    # Ensure consistent length
                    if aug_seq.shape[0] != self.max_seq_len:
                        sequence = self._pad_or_truncate(aug_seq)
                    else:
                        sequence = aug_seq
        
        return torch.FloatTensor(sequence), torch.LongTensor([label])[0]

print("✅ Enhanced ASLKeypointDataset with 'nodes' key support and validation ready!")

✅ Enhanced ASLKeypointDataset with 'nodes' key support and validation ready!


## Transformer Model Architecture


In [42]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class TransformerASL(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, num_classes, seq_len, dropout=0.1):
        super(TransformerASL, self).__init__()
        
        self.input_projection = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, seq_len)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True
        )
        
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, num_classes)
        )
        
        self.d_model = d_model
    
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        x = self.input_projection(x) * np.sqrt(self.d_model)
        x = x.transpose(0, 1)  # (seq_len, batch_size, d_model)
        x = self.pos_encoder(x)
        x = x.transpose(0, 1)  # (batch_size, seq_len, d_model)
        
        # Create padding mask
        src_key_padding_mask = torch.all(x == 0, dim=-1)
        
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)
        
        # Global average pooling
        mask = (~src_key_padding_mask).float().unsqueeze(-1)
        x = (x * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1)
        
        return self.classifier(x)

## 🛠️ Metrics Tracking and Training Utilities

Advanced metrics tracking with visualizations and training utilities from TGCN pipeline.


In [43]:
class MetricsTracker:
    """Track training metrics and progress with advanced visualizations"""
    
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.train_losses = []
        self.train_accuracies = []
        self.val_losses = []
        self.val_accuracies = []
        self.learning_rates = []
        self.best_val_acc = 0.0
        self.best_epoch = 0
        self.epoch_times = []
        self.samples_per_second = []
    
    def update(self, train_loss, train_acc, val_loss, val_acc, lr, epoch_time=None, samples_per_sec=None):
        self.train_losses.append(train_loss)
        self.train_accuracies.append(train_acc)
        self.val_losses.append(val_loss)
        self.val_accuracies.append(val_acc)
        self.learning_rates.append(lr)
        
        if epoch_time is not None:
            self.epoch_times.append(epoch_time)
        if samples_per_sec is not None:
            self.samples_per_second.append(samples_per_sec)
        
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            self.best_epoch = len(self.val_accuracies) - 1
    
    def plot_metrics(self):
        """Plot comprehensive training metrics"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        
        # Loss plot
        axes[0, 0].plot(self.train_losses, label='Train Loss', color='blue', linewidth=2)
        axes[0, 0].plot(self.val_losses, label='Val Loss', color='red', linewidth=2)
        axes[0, 0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Accuracy plot
        axes[0, 1].plot(self.train_accuracies, label='Train Acc', color='blue', linewidth=2)
        axes[0, 1].plot(self.val_accuracies, label='Val Acc', color='red', linewidth=2)
        axes[0, 1].axhline(y=self.best_val_acc, color='green', linestyle='--', 
                          label=f'Best Val Acc: {self.best_val_acc:.3f}%', linewidth=2)
        axes[0, 1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Accuracy (%)')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Learning rate plot
        axes[0, 2].plot(self.learning_rates, color='orange', linewidth=2)
        axes[0, 2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
        axes[0, 2].set_xlabel('Epoch')
        axes[0, 2].set_ylabel('Learning Rate')
        axes[0, 2].set_yscale('log')
        axes[0, 2].grid(True, alpha=0.3)
        
        # Validation accuracy zoomed
        axes[1, 0].plot(self.val_accuracies, color='red', linewidth=2, marker='o', markersize=4)
        axes[1, 0].axhline(y=self.best_val_acc, color='green', linestyle='--', linewidth=2)
        axes[1, 0].set_title(f'Validation Accuracy\n(Best: {self.best_val_acc:.3f}% at epoch {self.best_epoch+1})', 
                            fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Accuracy (%)')
        axes[1, 0].grid(True, alpha=0.3)
        
        # Training speed
        if self.epoch_times:
            axes[1, 1].plot(self.epoch_times, color='purple', linewidth=2)
            axes[1, 1].set_title('Training Speed (Time per Epoch)', fontsize=14, fontweight='bold')
            axes[1, 1].set_xlabel('Epoch')
            axes[1, 1].set_ylabel('Time (seconds)')
            axes[1, 1].grid(True, alpha=0.3)
        else:
            axes[1, 1].text(0.5, 0.5, 'No timing data', ha='center', va='center', transform=axes[1, 1].transAxes)
            axes[1, 1].set_title('Training Speed (Time per Epoch)', fontsize=14, fontweight='bold')
        
        # Samples per second
        if self.samples_per_second:
            axes[1, 2].plot(self.samples_per_second, color='green', linewidth=2)
            axes[1, 2].set_title('Training Throughput', fontsize=14, fontweight='bold')
            axes[1, 2].set_xlabel('Epoch')
            axes[1, 2].set_ylabel('Samples/Second')
            axes[1, 2].grid(True, alpha=0.3)
        else:
            axes[1, 2].text(0.5, 0.5, 'No throughput data', ha='center', va='center', transform=axes[1, 2].transAxes)
            axes[1, 2].set_title('Training Throughput', fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        plt.show()

def calculate_accuracy(outputs, targets):
    """Calculate accuracy from model outputs and targets"""
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == targets).sum().item()
    total = targets.size(0)
    return 100.0 * correct / total

def train_epoch_with_progress(model, train_loader, criterion, optimizer, device, 
                             use_mixed_precision=False, scaler=None):
    """Train for one epoch with tqdm progress bar and optimizations"""
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    total_samples = 0
    start_time = time.time()
    
    # Create progress bar for batches
    train_pbar = tqdm(train_loader, desc='🚀 Training', leave=False, 
                     bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')
    
    for batch_idx, (data, targets) in enumerate(train_pbar):
        batch_start = time.time()
        
        # Move to device with non-blocking transfer if supported
        if torch.cuda.is_available():
            data = data.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
        else:
            data, targets = data.to(device), targets.to(device)
        
        # Reshape data for transformer (batch_size, seq_len, features)
        batch_size, seq_len, num_nodes, num_features = data.shape
        data = data.view(batch_size, seq_len, num_nodes * num_features)
        
        optimizer.zero_grad()
        
        # Mixed precision training
        if use_mixed_precision and scaler is not None:
            with autocast():
                outputs = model(data)
                loss = criterion(outputs, targets)
            
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(data)
            loss = criterion(outputs, targets)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
        
        total_loss += loss.item()
        accuracy = calculate_accuracy(outputs, targets)
        total_accuracy += accuracy
        num_batches += 1
        total_samples += batch_size
        
        # Update progress bar
        train_pbar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Acc': f'{accuracy:.2f}%',
            'AvgLoss': f'{total_loss/num_batches:.4f}'
        })
    
    # Calculate training speed
    total_time = time.time() - start_time
    samples_per_second = total_samples / total_time if total_time > 0 else 0
    
    return total_loss / num_batches, total_accuracy / num_batches, samples_per_second

def validate_epoch_with_progress(model, val_loader, criterion, device):
    """Validate for one epoch with tqdm progress bar"""
    model.eval()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    # Create progress bar for validation
    val_pbar = tqdm(val_loader, desc='🔍 Validation', leave=False,
                   bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}, {rate_fmt}]')
    
    with torch.no_grad():
        for data, targets in val_pbar:
            # Move to device
            if torch.cuda.is_available():
                data = data.to(device, non_blocking=True)
                targets = targets.to(device, non_blocking=True)
            else:
                data, targets = data.to(device), targets.to(device)
            
            # Reshape data for transformer
            batch_size, seq_len, num_nodes, num_features = data.shape
            data = data.view(batch_size, seq_len, num_nodes * num_features)
            
            outputs = model(data)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
            accuracy = calculate_accuracy(outputs, targets)
            total_accuracy += accuracy
            num_batches += 1
            
            # Update progress bar
            val_pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{accuracy:.2f}%',
                'AvgAcc': f'{total_accuracy/num_batches:.2f}%'
            })
    
    return total_loss / num_batches, total_accuracy / num_batches

print("✅ Advanced metrics tracking and training utilities ready!")

✅ Advanced metrics tracking and training utilities ready!


## Data Loading and Preparation


In [44]:

# Data paths and directories
DATA_DIR = r'f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints'
CHECKPOINT_DIR = r'f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\src\checkpoints'
MODEL_SAVE_PATH = os.path.join(CHECKPOINT_DIR, 'best_transformer_model.pth')

# Create directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Fix the dataset creation logic
print("🔄 Creating enhanced datasets with proper train/val/test splits...")

# Create normalizer
normalizer = SimpleNormalizer()

# First, create a temporary dataset to get all data and fit normalizer
temp_dataset = ASLKeypointDataset(
    data_dir=DATA_DIR,
    max_seq_len=CONFIG['max_seq_len'],
    split='train',  # Get training portion for normalization
    test_size=CONFIG['test_size'],
    random_state=42,
    use_subset=True,
    max_classes=CONFIG['max_classes'],
    normalizer=None,
    use_augmentation=False
)

# Fit normalizer on training data
print("🔧 Fitting normalizer on training data...")
normalizer.fit([temp_dataset.data[i] for i in range(len(temp_dataset.data))])

# Now create the actual datasets with proper splits
# Create training dataset
train_dataset = ASLKeypointDataset(
    data_dir=DATA_DIR,
    max_seq_len=CONFIG['max_seq_len'],
    split='train',
    test_size=CONFIG['test_size'],
    random_state=42,
    use_subset=True,
    max_classes=CONFIG['max_classes'],
    normalizer=normalizer,
    use_augmentation=CONFIG['use_augmentation']
)

# Create test dataset (this will be our validation set)
test_dataset = ASLKeypointDataset(
    data_dir=DATA_DIR,
    max_seq_len=CONFIG['max_seq_len'],
    split='test',  # This gives us the test portion
    test_size=CONFIG['test_size'],
    random_state=42,
    use_subset=True,
    max_classes=CONFIG['max_classes'],
    normalizer=normalizer,
    use_augmentation=False
)

# For validation, we'll use a portion of test data
# Split test data into val and test
val_size = len(test_dataset) // 2  # Use half for validation, half for testing
test_size = len(test_dataset) - val_size

val_dataset, final_test_dataset = random_split(
    test_dataset, 
    [val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"✅ Datasets created successfully!")
print(f"📊 Training samples: {len(train_dataset)}")
print(f"📊 Validation samples: {len(val_dataset)}")
print(f"📊 Test samples: {len(final_test_dataset)}")
print(f"📊 Number of classes: {train_dataset.num_classes}")

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=CONFIG['pin_memory'] if torch.cuda.is_available() else False,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=CONFIG['pin_memory'] if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    final_test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=CONFIG['pin_memory'] if torch.cuda.is_available() else False
)

# Get class names from training dataset
class_names = train_dataset.class_names

# Save class mapping
class_mapping = {
    'word_to_idx': train_dataset.word_to_idx,
    'idx_to_word': train_dataset.idx_to_word,
    'class_names': train_dataset.class_names
}

with open(os.path.join(CHECKPOINT_DIR, 'class_mapping.json'), 'w') as f:
    json.dump(class_mapping, f, indent=2)

print(f"💾 Class mapping saved to {CHECKPOINT_DIR}/class_mapping.json")

# Test data loader
print("\n🧪 Testing data loader...")
test_batch = next(iter(train_loader))
test_data, test_targets = test_batch
print(f"  Batch data shape: {test_data.shape}")
print(f"  Batch targets shape: {test_targets.shape}")
print(f"  Data type: {test_data.dtype}")
print(f"  Target range: {test_targets.min().item()} to {test_targets.max().item()}")

🔄 Creating enhanced datasets with proper train/val/test splits...
🔄 Loading train dataset from f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints...
  📊 Using subset: 50 classes


Loading classes:   0%|          | 0/50 [00:00<?, ?it/s]

  📁 about: 8 valid samples
  📁 accident: 13 valid samples
  📁 africa: 13 valid samples
  📁 again: 10 valid samples
  📁 all: 13 valid samples
  📁 always: 9 valid samples
  📁 always: 9 valid samples
  📁 animal: 10 valid samples
  📁 apple: 13 valid samples
  📁 approve: 11 valid samples
  📁 argue: 10 valid samples
  📁 arrive: 10 valid samples
  📁 baby: 10 valid samples
  📁 back: 7 valid samples
  📁 animal: 10 valid samples
  📁 apple: 13 valid samples
  📁 approve: 11 valid samples
  📁 argue: 10 valid samples
  📁 arrive: 10 valid samples
  📁 baby: 10 valid samples
  📁 back: 7 valid samples
  📁 backpack: 11 valid samples
  📁 bad: 11 valid samples
  📁 backpack: 11 valid samples
  📁 bad: 11 valid samples
  📁 bake: 8 valid samples
  📁 balance: 11 valid samples
  📁 ball: 11 valid samples
  📁 banana: 10 valid samples
  📁 bar: 10 valid samples
  📁 basketball: 12 valid samples
  📁 bake: 8 valid samples
  📁 balance: 11 valid samples
  📁 ball: 11 valid samples
  📁 banana: 10 valid samples
  📁 bar: 10 

Processing sequences for normalization:   0%|          | 0/425 [00:00<?, ?it/s]

  📊 Valid keypoints: 10,784,367 / 11,751,250 (91.8%)
  📈 Normalization parameters:
    Center: (0.506, 0.237)
    Scale: 0.445
🔄 Loading train dataset from f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints...
  📊 Using subset: 50 classes
  📈 Normalization parameters:
    Center: (0.506, 0.237)
    Scale: 0.445
🔄 Loading train dataset from f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints...
  📊 Using subset: 50 classes


Loading classes:   0%|          | 0/50 [00:00<?, ?it/s]

  📁 about: 8 valid samples
  📁 accident: 13 valid samples
  📁 africa: 13 valid samples
  📁 again: 10 valid samples
  📁 all: 13 valid samples
  📁 all: 13 valid samples
  📁 always: 9 valid samples
  📁 animal: 10 valid samples
  📁 apple: 13 valid samples
  📁 approve: 11 valid samples
  📁 argue: 10 valid samples
  📁 always: 9 valid samples
  📁 animal: 10 valid samples
  📁 apple: 13 valid samples
  📁 approve: 11 valid samples
  📁 argue: 10 valid samples
  📁 arrive: 10 valid samples
  📁 arrive: 10 valid samples
  📁 baby: 10 valid samples
  📁 back: 7 valid samples
  📁 backpack: 11 valid samples
  📁 bad: 11 valid samples
  📁 bake: 8 valid samples
  📁 balance: 11 valid samples
  📁 baby: 10 valid samples
  📁 back: 7 valid samples
  📁 backpack: 11 valid samples
  📁 bad: 11 valid samples
  📁 bake: 8 valid samples
  📁 balance: 11 valid samples
  📁 ball: 11 valid samples
  📁 banana: 10 valid samples
  📁 bar: 10 valid samples
  📁 basketball: 12 valid samples
  📁 bath: 10 valid samples
  📁 ball: 11 va

Loading classes:   0%|          | 0/50 [00:00<?, ?it/s]

  📁 about: 8 valid samples
  📁 accident: 13 valid samples
  📁 africa: 13 valid samples
  📁 again: 10 valid samples
  📁 again: 10 valid samples
  📁 all: 13 valid samples
  📁 always: 9 valid samples
  📁 animal: 10 valid samples
  📁 apple: 13 valid samples
  📁 all: 13 valid samples
  📁 always: 9 valid samples
  📁 animal: 10 valid samples
  📁 apple: 13 valid samples
  📁 approve: 11 valid samples
  📁 approve: 11 valid samples
  📁 argue: 10 valid samples
  📁 arrive: 10 valid samples
  📁 baby: 10 valid samples
  📁 back: 7 valid samples
  📁 backpack: 11 valid samples
  📁 bad: 11 valid samples
  📁 argue: 10 valid samples
  📁 arrive: 10 valid samples
  📁 baby: 10 valid samples
  📁 back: 7 valid samples
  📁 backpack: 11 valid samples
  📁 bad: 11 valid samples
  📁 bake: 8 valid samples
  📁 bake: 8 valid samples
  📁 balance: 11 valid samples
  📁 ball: 11 valid samples
  📁 banana: 10 valid samples
  📁 bar: 10 valid samples
  📁 basketball: 12 valid samples
  📁 balance: 11 valid samples
  📁 ball: 11 v

## 🚀 Ultra-Optimized Model Initialization and Training Setup

Initialize the Transformer model with all GPU optimizations and training enhancements.


In [45]:
# Add the missing comprehensive_evaluation function
def comprehensive_evaluation(model, data_loader, device, class_names):
    """Comprehensive model evaluation with detailed metrics"""
    model.eval()
    all_predictions = []
    all_targets = []
    total_loss = 0.0
    num_batches = 0
    
    print("🔄 Running comprehensive evaluation...")
    
    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(tqdm(data_loader, desc="Evaluating")):
            data, targets = data.to(device), targets.to(device)
            
            # Handle data reshaping for transformer input
            if len(data.shape) == 4:  # (batch, seq_len, nodes, features)
                batch_size, seq_len, num_nodes, num_features = data.shape
                data = data.view(batch_size, seq_len, num_nodes * num_features)
            
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            num_batches += 1
    
    # Calculate accuracy
    accuracy = accuracy_score(all_targets, all_predictions)
    
    # Generate classification report
    report = classification_report(
        all_targets, all_predictions, 
        target_names=class_names, 
        output_dict=True,
        zero_division=0
    )
    
    # Print detailed results
    print(f"\n📊 Evaluation Results:")
    print(f"  🎯 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  📈 Macro F1-Score: {report['macro avg']['f1-score']:.4f}")
    print(f"  📊 Weighted F1-Score: {report['weighted avg']['f1-score']:.4f}")
    
    # Print per-class results (first 10 classes)
    print(f"\n📋 Per-Class Performance (Top 10):")
    print(f"{'Class':<15} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
    print("-" * 55)
    
    for i, class_name in enumerate(class_names[:10]):
        if class_name in report:
            p = report[class_name]['precision']
            r = report[class_name]['recall']
            f1 = report[class_name]['f1-score']
            print(f"{class_name:<15} {p:<10.3f} {r:<10.3f} {f1:<10.3f}")
    
    if len(class_names) > 10:
        print(f"... and {len(class_names) - 10} more classes")
    
    return accuracy, report

print("🚀 Initializing Ultra-Optimized Transformer Model")
print("=" * 60)

# Get input dimension from CONFIG (already calculated)
input_dim = CONFIG['input_dim']

# Initialize model
model = TransformerASL(
    input_dim=input_dim,
    d_model=CONFIG['d_model'],
    nhead=CONFIG['nhead'],
    num_layers=CONFIG['num_layers'],
    num_classes=train_dataset.num_classes,
    seq_len=CONFIG['max_seq_len'],
    dropout=CONFIG['dropout']
).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"🏗️ Model Architecture:")
print(f"  Input dimension: {input_dim}")
print(f"  Model dimension: {CONFIG['d_model']}")
print(f"  Attention heads: {CONFIG['nhead']}")
print(f"  Transformer layers: {CONFIG['num_layers']}")
print(f"  Output classes: {train_dataset.num_classes}")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Model size: {total_params * 4 / 1024 / 1024:.2f} MB")

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay'],
    betas=(0.9, 0.999),
    eps=1e-8
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=CONFIG['learning_rate'],
    epochs=CONFIG['num_epochs'],
    steps_per_epoch=len(train_loader),
    pct_start=0.1,
    anneal_strategy='cos'
)

# Initialize mixed precision scaler if enabled
scaler = None
if CONFIG['use_mixed_precision'] and torch.cuda.is_available():
    scaler = GradScaler()
    print(f"✅ Mixed precision training enabled")

print(f"\n⚙️ Training Configuration:")
print(f"  Device: {device}")
print(f"  Optimizer: AdamW with OneCycleLR")
print(f"  Loss: CrossEntropyLoss with label smoothing")
print(f"  Mixed precision: {CONFIG['use_mixed_precision']}")
print(f"  Data augmentation: {CONFIG['use_augmentation']}")
print(f"  Batch size: {CONFIG['batch_size']}")
print(f"  Max epochs: {CONFIG['num_epochs']}")

# Test forward pass
print(f"\n🧪 Testing model forward pass...")
model.eval()
with torch.no_grad():
    test_batch = next(iter(train_loader))
    test_data, test_targets = test_batch[0][:2].to(device), test_batch[1][:2].to(device)
    
    # Reshape for transformer
    batch_size, seq_len, num_nodes, num_features = test_data.shape
    test_data_reshaped = test_data.view(batch_size, seq_len, num_nodes * num_features)
    
    test_output = model(test_data_reshaped)
    print(f"  Input shape: {test_data.shape}")
    print(f"  Reshaped input: {test_data_reshaped.shape}")
    print(f"  Output shape: {test_output.shape}")
    print(f"  Expected output: [2, {train_dataset.num_classes}]")
    
    if test_output.shape == (2, train_dataset.num_classes):
        print(f"  ✅ Forward pass successful!")
    else:
        print(f"  ❌ Shape mismatch in forward pass!")

print("=" * 60)
print("🎯 Ready for ultra-optimized training!")

🚀 Initializing Ultra-Optimized Transformer Model
🏗️ Model Architecture:
  Input dimension: 1659
  Model dimension: 160
  Attention heads: 10
  Transformer layers: 8
  Output classes: 50
  Total parameters: 2,756,770
  Trainable parameters: 2,756,770
  Model size: 10.52 MB
✅ Mixed precision training enabled

⚙️ Training Configuration:
  Device: cuda
  Optimizer: AdamW with OneCycleLR
  Loss: CrossEntropyLoss with label smoothing
  Mixed precision: True
  Data augmentation: True
  Batch size: 12
  Max epochs: 100

🧪 Testing model forward pass...
  Input shape: torch.Size([2, 50, 553, 3])
  Reshaped input: torch.Size([2, 50, 1659])
  Output shape: torch.Size([2, 50])
  Expected output: [2, 50]
  ✅ Forward pass successful!
🎯 Ready for ultra-optimized training!


## 🚀 Ultra-Optimized Training Pipeline

Implementing the complete ultra-optimized training pipeline with all advanced features from TGCN pipeline.


In [46]:
print("Starting training...")

# Training history
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

best_val_acc = 0
epochs_without_improvement = 0

def ultra_optimized_training_pipeline(model, train_loader, val_loader, 
                                      optimizer, scheduler, scaler, criterion,
                                      device, config, class_names):
    """
    Ultra-optimized training pipeline with all advanced features:
    - Mixed precision training
    - Progress bars with detailed metrics
    - Model checkpointing
    - Advanced learning rate scheduling
    - Comprehensive metrics tracking
    """
    
    print("🚀 Starting Ultra-Optimized Training Pipeline...")
    print(f"📊 Configuration: {len(class_names)} classes, {config['num_epochs']} epochs")
    print(f"⚙️  Mixed Precision: {config['use_mixed_precision']}")
    print(f"💾 Checkpointing: {config['save_checkpoints']}")
    
    # Initialize metrics tracker
    metrics = MetricsTracker()
    
    # Create checkpoint directory
    checkpoint_dir = config.get('checkpoint_dir', 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Training loop with progress tracking
    best_val_acc = 0.0
    epochs_without_improvement = 0
    
    # Main training epochs with tqdm
    epoch_pbar = trange(config['num_epochs'], desc="🏋️  Training Progress", 
                       unit="epoch", position=0)
    
    for epoch in epoch_pbar:
        start_time = time.time()
        
        # Training phase - Fixed parameter passing
        train_loss, train_acc, train_speed = train_epoch_with_progress(
            model, train_loader, criterion, optimizer, device, 
            config['use_mixed_precision'], scaler)
        
        # Validation phase  
        val_loss, val_acc = validate_epoch_with_progress(
            model, val_loader, criterion, device)
        
        
        # Scheduler step
        if config['scheduler_type'] == 'ReduceLROnPlateau':
            scheduler.step(val_loss)
        else:
            scheduler.step()
        
        # Calculate epoch time and speed
        epoch_time = time.time() - start_time
        current_lr = optimizer.param_groups[0]['lr']
        
        # Update metrics
        metrics.update(train_loss, train_acc, val_loss, val_acc, current_lr, 
                      epoch_time, train_speed)
        
        # Check for best model
        is_best = val_acc > best_val_acc
        if is_best:
            best_val_acc = val_acc
            epochs_without_improvement = 0
            
            # Save best model
            if config['save_checkpoints']:
                best_model_path = os.path.join(checkpoint_dir, 'best_transformer_model.pth')
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'scaler_state_dict': scaler.state_dict() if scaler else None,
                    'val_acc': val_acc,
                    'train_acc': train_acc,
                    'val_loss': val_loss,
                    'train_loss': train_loss,
                    'class_names': class_names,
                    'config': config
                }, best_model_path)
        else:
            epochs_without_improvement += 1
        
        # Update progress bar description
        epoch_pbar.set_postfix({
            'Train_Acc': f'{train_acc:.1f}%',
            'Val_Acc': f'{val_acc:.1f}%', 
            'Best_Val': f'{best_val_acc:.1f}%',
            'LR': f'{current_lr:.2e}',
            'Speed': f'{train_speed:.0f} smp/s'
        })
        
        # Detailed epoch summary
        print(f"\n📈 Epoch {epoch+1:3d}/{config['num_epochs']} Summary:")
        print(f"  🏋️  Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
        print(f"  🔍 Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
        print(f"  ⚡ Speed: {train_speed:.0f} samples/sec, Time: {epoch_time:.1f}s")
        print(f"  📚 LR: {current_lr:.2e}, Best Val Acc: {best_val_acc:.2f}%")
        
        # Early stopping check
        if epochs_without_improvement >= config['patience']:
            print(f"\n🛑 Early stopping triggered after {epochs_without_improvement} epochs without improvement")
            break
        
        # Periodic checkpoint saving
        if config['save_checkpoints'] and (epoch + 1) % config.get('checkpoint_frequency', 10) == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'metrics': metrics,
                'config': config
            }, checkpoint_path)
            print(f"  💾 Checkpoint saved: {checkpoint_path}")
    
    print(f"\n🎯 Training completed! Best validation accuracy: {best_val_acc:.2f}%")
    
    # Plot final metrics
    print("\n📊 Generating training visualizations...")
    metrics.plot_metrics()
    
    return metrics, best_val_acc

# Execute ultra-optimized training
print("🔥 Initializing Ultra-Optimized Training Pipeline...")

# Run the training pipeline
training_metrics, final_best_acc = ultra_optimized_training_pipeline(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader, 
    optimizer=optimizer,
    scheduler=scheduler,
    scaler=scaler,
    criterion=criterion,
    device=device,
    config=CONFIG,
    class_names=class_names
)

print(f"\n🏆 Training Pipeline Completed!")
print(f"📊 Final Best Validation Accuracy: {final_best_acc:.2f}%")

Starting training...
🔥 Initializing Ultra-Optimized Training Pipeline...
🚀 Starting Ultra-Optimized Training Pipeline...
📊 Configuration: 50 classes, 100 epochs
⚙️  Mixed Precision: True
💾 Checkpointing: True


🏋️  Training Progress:   0%|          | 0/100 [00:00<?, ?epoch/s]

🚀 Training:   0%|          | 0/35 [00:00<?, ?it/s]

🔍 Validation:   0%|          | 0/5 [00:00, ?it/s]


📈 Epoch   1/100 Summary:
  🏋️  Train: Loss=3.9204, Acc=1.67%
  🔍 Val:   Loss=3.9060, Acc=0.00%
  ⚡ Speed: 231 samples/sec, Time: 2.0s
  📚 LR: 1.20e-05, Best Val Acc: 0.00%


...

🚀 Training:   0%|          | 0/35 [00:00<?, ?it/s]

🔍 Validation:   0%|          | 0/5 [00:00, ?it/s]


📈 Epoch  34/100 Summary:
  🏋️  Train: Loss=3.4465, Acc=18.57%
  🔍 Val:   Loss=3.7454, Acc=9.00%
  ⚡ Speed: 269 samples/sec, Time: 1.7s
  📚 LR: 1.87e-05, Best Val Acc: 12.33%


🚀 Training:   0%|          | 0/35 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 🎯 Ultra-Comprehensive Model Evaluation

Loading best model and performing detailed evaluation with advanced metrics.


In [37]:
# Load the best model checkpoint
checkpoint_path = os.path.join(CONFIG.get('checkpoint_dir', 'checkpoints'), 'best_transformer_model.pth')

if os.path.exists(checkpoint_path):
    print(f"📥 Loading best model from: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"✅ Model loaded successfully!")
    print(f"  📊 Best epoch: {checkpoint['epoch'] + 1}")
    print(f"  🎯 Best validation accuracy: {checkpoint['val_acc']:.2f}%")
    print(f"  🏋️  Training accuracy: {checkpoint['train_acc']:.2f}%")
else:
    print(f"⚠️  No checkpoint found at {checkpoint_path}, using current model state")

# Perform comprehensive evaluation
print("\n🔍 Starting comprehensive model evaluation...")

# Test the model on validation set (since we used it for model selection)
print("\n📊 Validation Set Evaluation:")
val_accuracy, val_report = comprehensive_evaluation(model, val_loader, device, class_names)

# If test loader is available, evaluate on test set
if 'test_loader' in locals() and test_loader is not None:
    print("\n🧪 Test Set Evaluation:")
    test_accuracy, test_report = comprehensive_evaluation(model, test_loader, device, class_names)
else:
    print("\n⚠️  Test loader not available, using validation set for final evaluation")
    test_accuracy = val_accuracy
    test_report = val_report

📥 Loading best model from: f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\src\checkpoints\best_transformer_model.pth
✅ Model loaded successfully!
  📊 Best epoch: 14
  🎯 Best validation accuracy: 8.33%
  🏋️  Training accuracy: 9.80%

🔍 Starting comprehensive model evaluation...

📊 Validation Set Evaluation:
🔄 Running comprehensive evaluation...


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: Number of classes, 43, does not match size of target_names, 50. Try specifying the labels parameter

## 📈 Detailed Performance Analysis

Comprehensive analysis of model performance, efficiency, and insights.


In [ ]:
# Additional performance analysis
print("\n🔬 Detailed Performance Analysis:")

# Model complexity analysis
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n🧠 Model Architecture Analysis:")
print(f"  📊 Total parameters: {total_params:,}")
print(f"  🎯 Trainable parameters: {trainable_params:,}")
print(f"  💾 Model size: ~{total_params * 4 / 1024 / 1024:.1f} MB")

# Training efficiency analysis
if 'training_metrics' in locals():
    avg_epoch_time = np.mean(training_metrics.epoch_times) if training_metrics.epoch_times else 0
    avg_samples_per_sec = np.mean(training_metrics.samples_per_second) if training_metrics.samples_per_second else 0
    
    print(f"\n⚡ Training Efficiency:")
    print(f"  ⏱️  Average epoch time: {avg_epoch_time:.1f} seconds")
    print(f"  🚀 Average throughput: {avg_samples_per_sec:.0f} samples/second")
    print(f"  🎯 Best validation accuracy: {training_metrics.best_val_acc:.2f}%")
    print(f"  📈 Achieved at epoch: {training_metrics.best_epoch + 1}")

# Class distribution analysis
if hasattr(train_dataset, 'labels'):
    class_counts = np.bincount(train_dataset.labels)
    print(f"\n📊 Dataset Class Distribution:")
    print(f"  📚 Total classes: {len(class_names)}")
    print(f"  📝 Total samples: {len(train_dataset)}")
    print(f"  📊 Average samples per class: {len(train_dataset) / len(class_names):.1f}")
    print(f"  📉 Min samples in class: {class_counts.min()}")
    print(f"  📈 Max samples in class: {class_counts.max()}")

# Performance comparison with configuration
print(f"\n⚙️  Configuration Impact:")
print(f"  🔧 Mixed precision: {CONFIG['use_mixed_precision']}")
print(f"  📦 Batch size: {CONFIG['batch_size']}")
print(f"  🧠 Model dimension: {CONFIG['d_model']}")
print(f"  🔄 Transformer layers: {CONFIG['num_layers']}")
print(f"  🎯 Attention heads: {CONFIG['nhead']}")

# Memory usage estimation
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    memory_allocated = torch.cuda.memory_allocated(device) / 1024**3  # GB
    memory_reserved = torch.cuda.memory_reserved(device) / 1024**3   # GB
    
    print(f"\n💾 GPU Memory Usage:")
    print(f"  📊 Allocated: {memory_allocated:.2f} GB")
    print(f"  📦 Reserved: {memory_reserved:.2f} GB")
    print(f"  🎯 Device: {torch.cuda.get_device_name(device)}")

## 📊 Confusion Matrix and Class-wise Analysis

Detailed confusion matrix and per-class performance analysis.


In [ ]:
# Generate predictions for confusion matrix
model.eval()
all_predictions = []
all_targets = []

print("🔄 Generating predictions for confusion matrix...")
with torch.no_grad():
    for batch_idx, (data, targets) in enumerate(tqdm(val_loader, desc="Evaluating")):
        data, targets = data.to(device), targets.to(device)
        
        # Handle data reshaping for transformer input
        if len(data.shape) == 4:  # (batch, seq_len, nodes, features)
            batch_size, seq_len, num_nodes, num_features = data.shape
            data = data.view(batch_size, seq_len, num_nodes * num_features)
        
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

# Create confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(all_targets, all_predictions)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
if len(class_names) > 10:
    # Show only first 10 classes for readability
    cm_subset = cm[:10, :10]
    class_subset = class_names[:10]
    title = f"Confusion Matrix (First 10 Classes out of {len(class_names)})"
else:
    cm_subset = cm
    class_subset = class_names
    title = f"Confusion Matrix (All {len(class_names)} Classes)"

sns.heatmap(cm_subset, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_subset, yticklabels=class_subset)
plt.title(title, fontsize=14, fontweight='bold')
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Per-class accuracy analysis
class_accuracies = cm.diagonal() / cm.sum(axis=1)
print(f"\n📊 Per-Class Accuracy Analysis:")
print(f"{'Class':<15} {'Accuracy':<10} {'Support':<10}")
print("-" * 40)

for i, (class_name, acc) in enumerate(zip(class_names, class_accuracies)):
    support = cm.sum(axis=1)[i]
    print(f"{class_name:<15} {acc:<10.3f} {support:<10.0f}")
    if i >= 9:  # Show first 10 classes
        break

if len(class_names) > 10:
    print(f"... and {len(class_names) - 10} more classes")

# Overall accuracy
overall_accuracy = np.trace(cm) / np.sum(cm)
print(f"\n🎯 Overall Accuracy: {overall_accuracy:.3f} ({overall_accuracy*100:.2f}%)")

## 🏆 Final Results Summary

Comprehensive summary of the ultra-optimized ASL Transformer model performance and achievements.


In [ ]:
print("\n" + "="*80)
print("🏆 ULTRA-OPTIMIZED ASL TRANSFORMER - FINAL RESULTS SUMMARY")
print("="*80)

# Model Performance Summary
print(f"\n🎯 MODEL PERFORMANCE:")
print(f"  🥇 Best Validation Accuracy: {final_best_acc:.2f}%")
print(f"  🔍 Final Test Accuracy: {test_accuracy * 100:.2f}%")
if 'test_report' in locals():
    print(f"  📊 Macro F1-Score: {test_report['macro avg']['f1-score']:.3f}")
    print(f"  📈 Weighted F1-Score: {test_report['weighted avg']['f1-score']:.3f}")

# Architecture Summary
print(f"\n🧠 MODEL ARCHITECTURE:")
print(f"  🔧 Architecture: Transformer Encoder")
print(f"  📊 Model Dimension: {CONFIG['d_model']}")
print(f"  🔄 Encoder Layers: {CONFIG['num_layers']}")
print(f"  🎯 Attention Heads: {CONFIG['nhead']}")
print(f"  📝 Total Parameters: {total_params:,}")
print(f"  🎓 Trainable Parameters: {trainable_params:,}")

# Dataset Summary
print(f"\n📚 DATASET SUMMARY:")
print(f"  📊 Total Classes: {len(class_names)}")
print(f"  📝 Training Samples: {len(train_dataset)}")
print(f"  🔍 Validation Samples: {len(val_dataset)}")
print(f"  📏 Sequence Length: {CONFIG['max_seq_len']}")
print(f"  🎯 Input Features: {train_dataset[0][0].shape[-1]} per keypoint")

# Training Configuration
print(f"\n⚙️  TRAINING CONFIGURATION:")
print(f"  🔥 Optimizer: {CONFIG['optimizer_type']}")
print(f"  📈 Learning Rate: {CONFIG['learning_rate']}")
print(f"  📦 Batch Size: {CONFIG['batch_size']}")
print(f"  🔄 Max Epochs: {CONFIG['num_epochs']}")
print(f"  ⏱️  Patience: {CONFIG['patience']}")
print(f"  🎯 Mixed Precision: {CONFIG['use_mixed_precision']}")

# Performance Optimizations
print(f"\n🚀 PERFORMANCE OPTIMIZATIONS:")
print(f"  ⚡ GPU Acceleration: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  🎮 GPU Device: {torch.cuda.get_device_name(device)}")
print(f"  🔧 Mixed Precision Training: {CONFIG['use_mixed_precision']}")
print(f"  📊 Data Augmentation: {CONFIG.get('use_augmentation', False)}")
print(f"  💾 Model Checkpointing: {CONFIG['save_checkpoints']}")
print(f"  📈 Progress Tracking: tqdm with detailed metrics")

# Training Efficiency
if 'training_metrics' in locals() and training_metrics.epoch_times:
    total_training_time = sum(training_metrics.epoch_times)
    avg_epoch_time = np.mean(training_metrics.epoch_times)
    avg_throughput = np.mean(training_metrics.samples_per_second) if training_metrics.samples_per_second else 0
    
    print(f"\n⏱️  TRAINING EFFICIENCY:")
    print(f"  🕒 Total Training Time: {total_training_time:.1f} seconds ({total_training_time/60:.1f} minutes)")
    print(f"  📊 Average Epoch Time: {avg_epoch_time:.1f} seconds")
    if avg_throughput > 0:
        print(f"  🚀 Average Throughput: {avg_throughput:.0f} samples/second")
    print(f"  🎯 Epochs to Best Model: {training_metrics.best_epoch + 1}")

# Feature Highlights
print(f"\n⭐ ADVANCED FEATURES IMPLEMENTED:")
print(f"  🔄 Positional Encoding for sequence modeling")
print(f"  🎯 Multi-head Self-Attention mechanism")
print(f"  📊 Advanced metrics tracking with visualizations")
print(f"  🛡️  Gradient clipping for training stability")
print(f"  📈 Learning rate scheduling (OneCycleLR)")
print(f"  🔍 Comprehensive evaluation with classification reports")
print(f"  💾 Automatic model checkpointing")
print(f"  ⚡ Mixed precision training for efficiency")
print(f"  📊 Real-time progress monitoring with tqdm")
print(f"  🛑 Early stopping for optimal training")

# Key Improvements
print(f"\n🔧 KEY IMPROVEMENTS FROM BASIC VERSION:")
print(f"  ✅ Fixed 'nodes' vs 'pose' key compatibility")
print(f"  ✅ Enhanced normalization with robust statistics")
print(f"  ✅ Advanced training pipeline with mixed precision")
print(f"  ✅ Comprehensive metrics tracking and visualization")
print(f"  ✅ GPU optimization and memory efficiency")
print(f"  ✅ Professional-grade model checkpointing")
print(f"  ✅ Real-time progress monitoring")
print(f"  ✅ Advanced learning rate scheduling")
print(f"  ✅ Comprehensive evaluation framework")

print(f"\n" + "="*80)
print(f"🎉 TRAINING COMPLETED SUCCESSFULLY! 🎉")
print(f"Best model saved with {final_best_acc:.2f}% validation accuracy")
print(f"All advanced features from TGCN pipeline successfully integrated!")
print(f"Ultra-optimized ASL Transformer ready for deployment! 🚀")
print("="*80)

## 💾 Model Export and Deployment Preparation

Preparing the trained model for deployment and future use.


In [ ]:
# Save final model state and metadata
# Fix the deployment section to handle missing normalization stats gracefully
# Save final model state and metadata
print("💾 Preparing model for deployment...")

# Create final model package
final_model_path = os.path.join(CONFIG.get('checkpoint_dir', 'checkpoints'), 'asl_transformer_final.pth')

# Safely get normalization stats
normalization_stats = {}
if hasattr(normalizer, 'mean') and normalizer.mean is not None:
    normalization_stats['mean'] = normalizer.mean.tolist()
if hasattr(normalizer, 'std') and normalizer.std is not None:
    normalization_stats['std'] = normalizer.std.tolist()
if hasattr(normalizer, 'center_x') and normalizer.center_x is not None:
    normalization_stats['center_x'] = float(normalizer.center_x)
    normalization_stats['center_y'] = float(normalizer.center_y)
    normalization_stats['scale'] = float(normalizer.scale)

deployment_info = {
    'model_state_dict': model.state_dict(),
    'model_config': {
        'input_dim': CONFIG['input_dim'],
        'd_model': CONFIG['d_model'], 
        'nhead': CONFIG['nhead'],
        'num_layers': CONFIG['num_layers'],
        'num_classes': len(class_names),
        'seq_len': CONFIG['max_seq_len'],
        'dropout': CONFIG['dropout']
    },
    'class_names': class_names,
    'class_mapping': {
        'word_to_idx': train_dataset.word_to_idx,
        'idx_to_word': train_dataset.idx_to_word
    },
    'performance_metrics': {
        'best_val_acc': final_best_acc,
        'test_acc': test_accuracy * 100 if 'test_accuracy' in locals() else 0.0,
        'total_params': total_params,
        'trainable_params': trainable_params
    },
    'training_config': CONFIG,
    'normalization_stats': normalization_stats
}

torch.save(deployment_info, final_model_path)
print(f"✅ Final model saved to: {final_model_path}")

# Export model architecture summary
print(f"\n📋 Model Architecture Summary:")
print(f"  Input Shape: (batch_size, {CONFIG['max_seq_len']}, {CONFIG['input_dim']})")
print(f"  Output Shape: (batch_size, {len(class_names)})")
print(f"  Model Size: {total_params * 4 / 1024 / 1024:.1f} MB")
print(f"  Inference Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

# Create inference example
print(f"\n🔮 Creating inference example...")
sample_input = torch.randn(1, CONFIG['max_seq_len'], CONFIG['input_dim']).to(device)
model.eval()
with torch.no_grad():
    sample_output = model(sample_input)
    predicted_class_idx = torch.argmax(sample_output, dim=1).item()
    predicted_class = class_names[predicted_class_idx]
    confidence = torch.softmax(sample_output, dim=1).max().item()

print(f"✅ Inference test successful!")
print(f"  Sample prediction: {predicted_class} (confidence: {confidence:.3f})")

print(f"\n🎯 Model is ready for deployment and inference!")